In [35]:
import numpy as np
import soundfile as sf
import librosa
import os

# Compute RMS
def compute_rms(x):
    return np.sqrt(np.mean(np.square(x)))

FOLDER = 'snr'
SERIES_FOLDER = 'post_3'
noise_file = 'reference_noise_post_2-1.wav'
signal_files = [f for f in os.listdir(f'{FOLDER}/{SERIES_FOLDER}') if f.endswith('.wav')]
top_db = 20.0

# Load noise and compute RMS
noise, sr = sf.read(f'{FOLDER}/{SERIES_FOLDER}/{noise_file}')
if noise.ndim > 1:
    noise = noise.mean(axis=1)
noise_rms = compute_rms(noise)
print(f"Noise RMS: {noise_rms:.6f}")

# Process each pluck
snr_values = []
for path in signal_files:
    if path == noise_file:
        continue
    sig, _ = sf.read(f'{FOLDER}/{SERIES_FOLDER}/{path}')
    if sig.ndim > 1:
        sig = sig.mean(axis=1)
    trimmed, _ = librosa.effects.trim(sig, top_db=top_db)
    total_rms = compute_rms(trimmed)
    signal_power = max(0.0, total_rms**2 - noise_rms**2)
    signal_rms = np.sqrt(signal_power)
    snr = signal_rms / noise_rms if noise_rms > 0 else np.inf
    snr_db = 20 * np.log10(signal_rms / noise_rms) if noise_rms > 0 else np.inf
    snr_values.append(snr_db)
    print(f"{path}: Signal RMS = {signal_rms:.6f}, SNR = {snr:.2f} | {snr_db:.2f} dB")

# Average SNR
avg_snr = np.mean(snr_values)
median_snr = np.median(snr_values)
dev_snr = np.std(snr_values)
print(f"Standard Deviation of SNR: {dev_snr:.2f} dB")
print(f"Median SNR: {median_snr:.2f} dB")
print(f"Average SNR: {avg_snr:.2f} dB")


Noise RMS: 0.001384
signal_post_2-3.wav: Signal RMS = 0.001144, SNR = 0.83 | -1.65 dB
signal_post_2-2.wav: Signal RMS = 0.000598, SNR = 0.43 | -7.29 dB
signal_post_2-1.wav: Signal RMS = 0.000855, SNR = 0.62 | -4.19 dB
Standard Deviation of SNR: 2.30 dB
Median SNR: -4.19 dB
Average SNR: -4.38 dB
